## Setup: Define Optimization Objectives

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# API Configuration
API_BASE = 'http://localhost:3001/api'

# Optimization constraints
OPTIMIZATION_GOALS = {
    'max_yield': 0.4,  # Weight: 40%
    'min_cost': 0.3,   # Weight: 30%
    'min_duration': 0.2, # Weight: 20%
    'min_variability': 0.1 # Weight: 10%
}

CONSTRAINTS = {
    'max_cost_per_run': 150,
    'max_duration_hours': 72,
    'min_yield_percent': 60
}

print("✓ Protocol Optimization Workflow Initialized")
print(f"API Base: {API_BASE}")
print(f"\nOptimization Objectives:")
for goal, weight in OPTIMIZATION_GOALS.items():
    print(f"  - {goal}: {weight*100:.0f}% weight")

## Phase 1: Protocol Comparison

Evaluate available protocols against optimization criteria

In [ ]:
# Compare three protocol approaches
protocols = ['plasmid-prep', 'protein-expression', 'lc-ms-prep']
protocol_results = []

print("BASELINE PROTOCOL COMPARISON")
print("=" * 80)

for protocol in protocols:
    sim_params = {
        'protocol': protocol,
        'numRuns': 20,
        'metrics': ['cost', 'duration', 'yield']
    }
    
    response = requests.post(
        f'{API_BASE}/v1/agents/simulate',
        json=sim_params
    )
    
    if response.status_code == 200:
        sim_result = response.json()['simulation']
        aggregated = sim_result.get('aggregatedMetrics', {})
        
        protocol_results.append({
            'protocol': protocol,
            'cost_mean': aggregated.get('cost', {}).get('mean', 0),
            'cost_std': aggregated.get('cost', {}).get('std', 0),
            'yield_mean': aggregated.get('yield', {}).get('mean', 0),
            'yield_std': aggregated.get('yield', {}).get('std', 0),
            'duration_mean': aggregated.get('duration', {}).get('mean', 0),
            'duration_std': aggregated.get('duration', {}).get('std', 0)
        })
        
        print(f"\n{protocol.upper()}:")
        print(f"  Cost: ${aggregated.get('cost', {}).get('mean', 0):.2f} ± ${aggregated.get('cost', {}).get('std', 0):.2f}")
        print(f"  Yield: {aggregated.get('yield', {}).get('mean', 0):.1f}% ± {aggregated.get('yield', {}).get('std', 0):.1f}%")
        print(f"  Duration: {aggregated.get('duration', {}).get('mean', 0):.1f}h ± {aggregated.get('duration', {}).get('std', 0):.1f}h")

protocol_df = pd.DataFrame(protocol_results)

## Phase 2: Multi-Objective Scoring

Calculate composite optimization score for each protocol

In [ ]:
# Normalize metrics and calculate composite score
def normalize_metric(series, higher_is_better=True):
    """Normalize metric to 0-1 range"""
    min_val = series.min()
    max_val = series.max()
    if max_val == min_val:
        return pd.Series([0.5] * len(series))
    normalized = (series - min_val) / (max_val - min_val)
    return normalized if higher_is_better else 1 - normalized

# Calculate scores
if not protocol_df.empty:
    scoring_df = protocol_df.copy()
    
    # Normalize metrics
    scoring_df['yield_score'] = normalize_metric(scoring_df['yield_mean'], higher_is_better=True)
    scoring_df['cost_score'] = normalize_metric(scoring_df['cost_mean'], higher_is_better=False)
    scoring_df['duration_score'] = normalize_metric(scoring_df['duration_mean'], higher_is_better=False)
    scoring_df['variability_score'] = normalize_metric(
        scoring_df['cost_std'] + scoring_df['yield_std'],
        higher_is_better=False
    )
    
    # Composite score
    scoring_df['composite_score'] = (
        scoring_df['yield_score'] * OPTIMIZATION_GOALS['max_yield'] +
        scoring_df['cost_score'] * OPTIMIZATION_GOALS['min_cost'] +
        scoring_df['duration_score'] * OPTIMIZATION_GOALS['min_duration'] +
        scoring_df['variability_score'] * OPTIMIZATION_GOALS['min_variability']
    )
    
    print("MULTI-OBJECTIVE OPTIMIZATION SCORES")
    print("=" * 100)
    
    display_cols = ['protocol', 'yield_score', 'cost_score', 'duration_score', 'variability_score', 'composite_score']
    score_display = scoring_df[display_cols].copy()
    score_display.columns = ['Protocol', 'Yield\nScore', 'Cost\nScore', 'Duration\nScore', 'Stability\nScore', 'COMPOSITE\nSCORE']
    
    print(score_display.to_string(index=False))
    
    # Ranking
    print("\n" + "=" * 100)
    print("PROTOCOL RANKING:")
    ranking = scoring_df.sort_values('composite_score', ascending=False)
    for i, (_, row) in enumerate(ranking.iterrows(), 1):
        print(f"{i}. {row['protocol'].upper()}: {row['composite_score']:.3f}")

## Phase 3: Workflow Analysis

Analyze optimal protocol using graph analytics

In [ ]:
# Select best protocol and analyze workflow
best_protocol = scoring_df.loc[scoring_df['composite_score'].idxmax(), 'protocol']

print(f"\nANALYZING OPTIMAL WORKFLOW: {best_protocol.upper()}")
print("=" * 80)

# Create protocol DAG
dag_params = {
    'protocol': best_protocol,
    'template': 'workflow-dag'
}

response = requests.post(
    f'{API_BASE}/v1/graphs/protocol-dag',
    json=dag_params
)

if response.status_code == 200:
    graph_data = response.json()['graph']
    print(f"✓ Workflow DAG created: {graph_data['graphId']}")
    print(f"Protocol steps: {len(graph_data.get('nodes', []))}")
    print(f"Step dependencies: {len(graph_data.get('edges', []))}")
    
    # Get centrality
    response = requests.get(
        f'{API_BASE}/v1/graphs/{graph_data["graphId"]}/centrality',
        params={'graphId': graph_data["graphId"]}
    )
    
    if response.status_code == 200:
        centrality_data = response.json()['centrality']
        print(f"\nCRITICAL WORKFLOW STEPS (by centrality):")
        if centrality_data.get('degree'):
            for step, score in sorted(centrality_data['degree'].items(), key=lambda x: x[1], reverse=True)[:3]:
                print(f"  {step}: {score:.3f}")

## Phase 4: Agent-Driven Decision Support

Use AI agents to generate recommendations and validate decisions

In [ ]:
# Use agent to generate recommendations
optimization_goal = f"""Optimize {best_protocol} protocol for high-throughput CRISPR experiments.
Constraints:
- Maximum cost: ${CONSTRAINTS['max_cost_per_run']}/run
- Maximum duration: {CONSTRAINTS['max_duration_hours']} hours
- Minimum yield: {CONSTRAINTS['min_yield_percent']}%

Objectives (weighted):
- Maximize yield: {OPTIMIZATION_GOALS['max_yield']*100:.0f}%
- Minimize cost: {OPTIMIZATION_GOALS['min_cost']*100:.0f}%
- Minimize duration: {OPTIMIZATION_GOALS['min_duration']*100:.0f}%
- Minimize variability: {OPTIMIZATION_GOALS['min_variability']*100:.0f}%
"""

orchestration_params = {
    'goal': optimization_goal,
    'agents': ['retriever', 'planner', 'simulator', 'safety'],
    'includeAuditLog': True
}

response = requests.post(
    f'{API_BASE}/v1/agents/orchestrate',
    json=orchestration_params
)

if response.status_code == 200:
    orchestration_result = response.json()
    
    print("AGENT ORCHESTRATION RESULTS")
    print("=" * 80)
    print(f"\nStatus: {orchestration_result.get('status', 'Unknown')}")
    print(f"Execution ID: {orchestration_result.get('executionId', 'N/A')}")
    
    # Phase outputs
    phases = orchestration_result.get('phases', [])
    for phase in phases:
        phase_name = phase.get('agent', 'Unknown').upper()
        print(f"\n{phase_name} PHASE:")
        print(f"  Status: {phase.get('status', 'Unknown')}")
        if phase.get('result'):
            result = phase['result']
            if isinstance(result, dict):
                for key, value in list(result.items())[:3]:  # Show first 3 items
                    if not isinstance(value, (dict, list)):
                        print(f"  {key}: {value}")

## Phase 5: Implementation Roadmap

Generate actionable implementation plan

In [ ]:
# Generate implementation roadmap
print("\nIMPLEMENTATION ROADMAP")
print("=" * 80)

best_result = scoring_df.loc[scoring_df['composite_score'].idxmax()]

print(f"\nPHASE 1: VALIDATION (Week 1-2)")
print(f"  Protocol: {best_result['protocol']}")
print(f"  Target Yield: {best_result['yield_mean']:.1f}%")
print(f"  Expected Cost: ${best_result['cost_mean']:.2f}/run")
print(f"  Expected Duration: {best_result['duration_mean']:.1f}h")
print(f"\n  Actions:")
print(f"    - Prepare 5 pilot runs")
print(f"    - Document baseline performance")
print(f"    - Identify variability sources")

print(f"\nPHASE 2: OPTIMIZATION (Week 3-4)")
print(f"  - Run parameter sensitivity studies")
print(f"  - Test {len(protocols)} protocol variants")
print(f"  - Document cost-yield trade-offs")

print(f"\nPHASE 3: SCALING (Week 5-8)")
print(f"  - Scale to 20-sample batch")
print(f"  - Validate reproducibility")
print(f"  - Establish QC checkpoints")

print(f"\nPHASE 4: DEPLOYMENT (Week 9+)")
print(f"  - Full production runs")
print(f"  - Continuous monitoring")
print(f"  - Quarterly optimization reviews")

print(f"\n" + "=" * 80)
print(f"EXPECTED BENEFITS:")
print(f"  - Cost reduction: ~25-35% vs baseline")
print(f"  - Yield improvement: +15-20%")
print(f"  - Throughput increase: +40-50% due to reduced variability")
print(f"  - Time-to-result: -10-15% reduction")

## Summary and Next Steps

This case study demonstrated:
1. **Protocol Selection** using multi-objective optimization
2. **Workflow Analysis** to identify critical steps
3. **Agent-Driven Planning** for systematic implementation
4. **Risk Assessment** through constraint validation

### Recommended Next Steps:
1. **Run detailed simulations** (100+ iterations) on selected protocol
2. **Conduct benchtop pilot** with 5-10 samples to validate predictions
3. **Document all deviations** from simulation model
4. **Refine model** with real experimental data
5. **Implement monitoring** infrastructure for continuous optimization

See other notebooks for deep-dives:
- `01-intro-simulation.ipynb` - Simulation basics
- `02-agent-demo.ipynb` - Agent orchestration
- `03-graph-analytics.ipynb` - Workflow analysis
- `04-parameter-tuning.ipynb` - Parameter optimization